## Working With Text Data

The goal of this guide is to explore some of the main scikit-learn tools on a single practical task: analysing a collection of text documents (newsgroups posts) on twenty different topics.

In [1]:
import numpy as np
import pandas as pd

#### Import the data, limiting the selection to 4 classes

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [4]:
type(twenty_train)

sklearn.utils.Bunch

The returned dataset is a scikit-learn “bunch”: 
a simple holder object with fields that can be both accessed as python dict keys or object attributes for convenience, 
for instance the target_names holds the list of the requested category names:

In [5]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

The files themselves are loaded in memory in the data attribute. 
For reference the filenames are also available:

In [6]:
len(twenty_train.data)

2257

In [7]:
len(twenty_train.filenames)

2257

In [8]:
print("\n".join(twenty_train.data[0].split("\n")))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



For speed and space efficiency reasons scikit-learn loads the target attribute 
as an array of integers that corresponds to the index of the category name in the target_names list. 
The category integer id of each sample is stored in the target attribute:

In [9]:
print(twenty_train.target[0])

1


In [10]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


### Extracting features from text files
In order to perform machine learning on text documents, we first need to turn the text content into numerical feature vectors.

#### Bags of words
The most intuitive way to do so is the bags of words representation:

1. assign a fixed integer id to each word occurring in any document of the training set (for instance by building a dictionary from words to integer indices).
2. for each document #i, count the number of occurrences of each word w and store it in X[i, j] as the value of feature #j where j is the index of word w in the dictionary
The bags of words representation implies that n_features is the number of distinct words in the corpus: this number is typically larger than 100,000.

If n_samples == 10000, storing X as a numpy array of type float32 would require 10000 x 100000 x 4 bytes = 4GB in RAM which is barely manageable on today’s computers.

Fortunately, most values in X will be zeros since for a given document less than a couple thousands of distinct words will be used. For this reason we say that bags of words are typically high-dimensional sparse datasets. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.

scipy.sparse matrices are data structures that do exactly this, and scikit-learn has built-in support for these structures.

#### Tokenizing text with scikit-learn
Text preprocessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to feature vectors:

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [12]:
list(count_vect.vocabulary_)[50]

'email'

In [13]:
len(count_vect.vocabulary_)

35788

shape = #docs * #words in vodabulary (=features)

In [14]:
X_train_counts.shape

(2257, 35788)

For instance I look for the word Michael  in the 1st doc

In [15]:
list(count_vect.vocabulary_).index('michael')

5

In [16]:
(X_train_counts[0].toarray()).shape

(1, 35788)

In [17]:
pd.DataFrame(data=X_train_counts[0].toarray(), columns=count_vect.get_feature_names()).loc[:,'michael']

0    3
Name: michael, dtype: int64

#### TF-IDF
Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called tf for Term Frequencies.

Another refinement on top of tf is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called tf–idf for “Term Frequency times Inverse Document Frequency”.

Both tf and tf–idf can be computed as follows:

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(2257, 35788)

In [19]:
pd.DataFrame(data=X_train_tf[0].toarray(), columns=count_vect.get_feature_names()).loc[:,'michael']

0    0.196228
Name: michael, dtype: float64

In the above example-code, we firstly use the fit(..) method to fit our estimator to the data and secondly the transform(..) method to transform our count-matrix to a tf-idf representation. These two steps can be combined to achieve the same end result faster by skipping redundant processing. This is done through using the fit_transform(..) method as shown below, and as mentioned in the note in the previous section:

In [20]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [21]:
pd.DataFrame(data=X_train_tfidf[0].toarray(), columns=count_vect.get_feature_names()).loc[:,'michael']

0    0.196228
Name: michael, dtype: float64

### Training a classifier
Now that we have our features, we can train a classifier to try to predict the category of a post. 
Let’s start with a naïve Bayes classifier, which provides a nice baseline for this task. scikit-learn includes several variants of this classifier; 
the one most suitable for word counts is the multinomial variant:

In [22]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

To try to predict the outcome on a new document we need to extract the features 
using almost the same feature extracting chain as before. 
The difference is that we call transform instead of fit_transform on the transformers, 
since they have already been fit to the training set:

In [23]:
docs_new = ['God is loved by Michael', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
print(X_new_tfidf)

  (0, 21661)	0.528894557934
  (0, 20539)	0.722733640083
  (0, 18474)	0.161434421506
  (0, 15521)	0.33345674538
  (0, 7505)	0.246317191025
  (1, 32142)	0.0891122234242
  (1, 23790)	0.600483554876
  (1, 23733)	0.128412754679
  (1, 18474)	0.100340184397
  (1, 15628)	0.677839555469
  (1, 14048)	0.381384400353


In [24]:
predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is loved by Michael' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


### Building a pipeline
In order to make the vectorizer => transformer => classifier easier to work with, 
scikit-learn provides a Pipeline class that behaves like a compound classifier:

In [25]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),])

Train the model

In [26]:
text_clf.fit(twenty_train.data, twenty_train.target) 

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

Retest on the previous example

In [27]:
predicted = text_clf.predict(docs_new)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is loved by Michael' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


### Evaluation of the performance on the test set
Evaluating the predictive accuracy of the model is equally easy:

In [28]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)            

0.83488681757656458

I.e., we achieved 83.4% accuracy. Let’s see if we can do better with a linear support vector machine (SVM), 
which is widely regarded as one of the best text classification algorithms (although it’s also a bit slower than naïve Bayes). 
We can change the learner by just plugging a different classifier object into our pipeline:

In [29]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),])
text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.9127829560585885

scikit-learn further provides utilities for more detailed performance analysis of the results:

In [30]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [31]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)

As expected the confusion matrix shows that posts from the newsgroups on atheism and christian are 
more often confused for one another than with computer graphics.

### Parameter tuning using grid search
We’ve already encountered some parameters such as use_idf in the TfidfTransformer. 
Classifiers tend to have many parameters as well; e.g., MultinomialNB includes a smoothing parameter alpha 
and SGDClassifier has a penalty parameter alpha and configurable loss and penalty terms in the objective function 
(see the module documentation, or use the Python help function, to get a description of these).

Instead of tweaking the parameters of the various components of the chain, 
it is possible to run an exhaustive search of the best parameters on a grid of possible values. 
We try out all classifiers on either words or bigrams, with or without idf, 
and with a penalty parameter of either 0.01 or 0.001 for the linear SVM:

In [32]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
                  'tfidf__use_idf': (True, False),
                  'clf__alpha': (1e-2, 1e-3),}

Obviously, such an exhaustive search can be expensive. 
If we have multiple CPU cores at our disposal, we can tell the grid searcher to try 
these eight parameter combinations in parallel with the n_jobs parameter. 
If we give this parameter a value of -1, grid search will detect how many cores are installed and uses them all:

In [33]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

The grid search instance behaves like a normal scikit-learn model. 
Let’s perform the search on a smaller subset of the training data to speed up the computation:

In [34]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

The result of calling fit on a GridSearchCV object is a classifier that we can use to predict:

In [35]:
twenty_train.target_names[gs_clf.predict(['God is loved by Michael'])[0]]

'soc.religion.christian'

The object’s best_score_ and best_params_ attributes store the best mean score and 
the parameters setting corresponding to that score:

In [36]:
gs_clf.best_score_  

0.90000000000000002

In [37]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


A more detailed summary of the search is available at gs_clf.cv_results_.

The cv_results_ parameter can be easily imported into pandas as a DataFrame for further inspection.

In [38]:
gs_clf.cv_results_

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\depr

{'mean_fit_time': array([ 0.17193484,  0.57163874,  0.15445654,  0.49566595,  0.1557397 ,
         0.55804769,  0.15691408,  0.4833401 ]),
 'mean_score_time': array([ 0.05053496,  0.10751081,  0.0493861 ,  0.12650704,  0.0550828 ,
         0.12699334,  0.0512952 ,  0.10161082]),
 'mean_test_score': array([ 0.8775,  0.875 ,  0.765 ,  0.78  ,  0.9   ,  0.89  ,  0.7675,  0.81  ]),
 'mean_train_score': array([ 0.99374372,  1.        ,  0.94123886,  0.97623272,  1.        ,
         1.        ,  0.98499057,  1.        ]),
 'param_clf__alpha': masked_array(data = [0.01 0.01 0.01 0.01 0.001 0.001 0.001 0.001],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_tfidf__use_idf': masked_array(data = [True True False False True True False False],
              mask = [False False False False False False False False],
        fill_value = ?),
 'param_vect__ngram_range': masked_array(data = [(1, 1) (1, 2) (1, 1) (1, 2) (1, 1) (1, 2) (1, 1) (1, 2